In [65]:
from langchain_ollama import ChatOllama
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import os

from llama_index.readers.papers import PubmedReader
import json





In [66]:
llm = ChatOllama(
            model="llama3.2",
            temperature=0,
            verbose=True
        )

In [67]:
@tool
def get_articles(disease_name):
    """This tool fetches articles from pubmed for a given disease name"""
    loader = PubmedReader()
    text_list = []
    page_title = []
    journal_title = []
    url_list = []
    map_data = []
    
    documents = loader.load_data(search_query=disease_name, max_results=4)
    for doc in documents:
      text_list.append(doc.text)
      page_title.append(doc.metadata['Title of this paper'])
      journal_title.append(doc.metadata['Journal it was published in:'])
      url_list.append(doc.metadata['URL'])
      map_data.append({"title": doc.metadata['Title of this paper'], "journal": doc.metadata['Journal it was published in:'], "url": doc.metadata['URL']})
      
    
    return map_data

tools = [get_articles]

In [68]:
prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "just use the {input} and give the url, title and journal of the articles based on the tool function response, format it in readable format and title of the response should be 'Articles on {input}', avoid notes and other unnecessary information",
                ),
                ("user", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

In [69]:
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt    
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "Hypertension"})

print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_articles` with `{'disease_name': 'Hypertension'}`


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=11469939&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=11469938&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=11469930&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=11469926&db=pmc
[{'title': 'Cost-effectiveness of Azvudine for High-risk Outpatients with Mild-to-moderate Coronavirus Disease 2019 in China', 'journal': 'Infection and Drug Resistance', 'url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11469939/'}, {'title': 'Targeting fibrosis, mechanisms and cilinical trials', 'journal': 'Physiological Reports', 'url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11469938/'}, {'title': 'Complication Avoidance and Management in Ambulatory Spine Surgery', 'journal': 'Case Reports in Surgery', 'url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11469930/'